# Machine Learning Poisonong Attacks in Malware Detection

## Prestart
### Import libraries

In [1]:
import csv

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

### Files
Store the paths for our files in variables

In [2]:
file_arm_training = "files\\arm_datasets\\arm_training.csv"
file_arm_validation = "files\\arm_datasets\\arm_validation.csv"
file_arm_test = "files\\arm_datasets\\arm_test.csv"

## Creating the arm dataset
$ 8:1:1 $ - Training:Test:Validation

#### Reading in
In this state the csv contains the filename and the label

In [3]:
arm_training = pd.read_csv(file_arm_training, header=None, index_col=False)
arm_validation = pd.read_csv(file_arm_validation, header=None, index_col=False)
arm_test = pd.read_csv(file_arm_test, header=None, index_col=False)

#### Dataset
Creating the pure dataset, only features

In [4]:
dataset_arm_training = np.asarray(arm_training.drop(columns=arm_training.columns[-2:]))
dataset_arm_validation = np.asarray(arm_validation.drop(columns=arm_validation.columns[-2:]))
dataset_arm_test = np.asarray(arm_test.drop(columns=arm_test.columns[-2:]))

#### Labels for the datasets
Creating the labels for the datasets
*Malware* - $ 0 $, *Benign* - $ 1 $
-**Trainining**: 800 malware, 800 benign
-**Validation**: 100 malware, 100 benign
-**Test**: 100 malware, 100 benign

In [5]:
labels_arm_training = np.asarray(arm_training[arm_training.columns[-1]])
labels_arm_validation = np.asarray(arm_validation[arm_validation.columns[-1]])
labels_arm_test = np.asarray(arm_test[arm_test.columns[-1]])

#### Filenames
Getting the filenames separated just for the fun of it

In [6]:
names_arm_training = arm_training[arm_training.columns[-2]]
names_arm_validation = arm_validation[arm_validation.columns[-2]]
names_arm_test = arm_test[arm_test.columns[-2]]

#### Generate

## Models for arm Datasets
In this section we will build the models with the exact same struture
Neural network with one *hidden layer* with a $sigmoid$ *activation function*

### Model
Building, fitting and evaluating the Model with the datasets

In [7]:
base_model = keras.Sequential(
    [
        layers.Dense(1, input_shape=(131,), activation="sigmoid")
    ]
)
base_model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.BinaryAccuracy()],
                   run_eagerly=True)
base_model.fit(dataset_arm_training, labels_arm_training, epochs=10,
                validation_data=(dataset_arm_validation, labels_arm_validation))

weights = base_model.get_weights()
print("weights: ", weights)

[_,binary_accuracy_base] = base_model.evaluate(dataset_arm_test, labels_arm_test)
print(binary_accuracy_base)
base_model.save("models\\base_model")

Epoch 1/10
90/90 [==============================] - 2s 17ms/step - loss: 0.5338 - binary_accuracy: 0.7677 - val_loss: 0.3638 - val_binary_accuracy: 0.8361
Epoch 2/10
90/90 [==============================] - 1s 16ms/step - loss: 0.3142 - binary_accuracy: 0.8774 - val_loss: 0.2420 - val_binary_accuracy: 0.9111
Epoch 3/10
90/90 [==============================] - 1s 15ms/step - loss: 0.2277 - binary_accuracy: 0.9215 - val_loss: 0.1753 - val_binary_accuracy: 0.9556
Epoch 4/10
90/90 [==============================] - 1s 16ms/step - loss: 0.1743 - binary_accuracy: 0.9448 - val_loss: 0.1364 - val_binary_accuracy: 0.9639
Epoch 5/10
90/90 [==============================] - 1s 16ms/step - loss: 0.1386 - binary_accuracy: 0.9580 - val_loss: 0.1202 - val_binary_accuracy: 0.9639
Epoch 6/10
90/90 [==============================] - 1s 15ms/step - loss: 0.1145 - binary_accuracy: 0.9677 - val_loss: 0.1011 - val_binary_accuracy: 0.9611
Epoch 7/10
90/90 [==============================] - 1s 15ms/step - los

In [8]:
df_arm_malware_forpoison = pd.read_csv("files\\poison_data\\arm_malware_forpoison.csv", header=None, index_col=False)
df_arm_malware_forpoison = df_arm_malware_forpoison.drop(columns=df_arm_malware_forpoison.columns[-2:])
topredict = np.asarray([df_arm_malware_forpoison.iloc[0],])
print(topredict)

[[210  14   1   3   1   0   0   1   1   2   0   2   2   2   2   3   3   2
    1   1   1   1   2   3   0   0   2   0   3   0   0   2   0   2   0   0
    3   1   2   3   2   3   3   0   3   3   2   3   1   2   2   3   0   1
    3   0   3   2   3   1   3   1   3   3   1   1   3   2   3   1   1   2
    3   0   1   3   1   1   3   2   2   1   2   0   2   0   2   2   0   2
    2   1   2   0   0   1   0   0   0   1   3   3   2   2   1   1   2   3
    0   2   2   1   0   0   0   3   1   0   0   0   0   2   3   1   1   0
    3   1   1   0   3]]


In [9]:
[[predict_base]] = base_model.predict(topredict)
print(predict_base)

1/1 [==============================] - 0s 24ms/step
0.19168939


## POISON IT

### Files

In [10]:
file_poison_arm_BM = "files\\poison_data\\iterative\\poisoned_benign_malware_1000.csv"
#file_poison_arm_MB = "files\\poison_data\\poisoned_malware_benign.csv"

### Poisoned Dataset

In [11]:
poisoned_arm_training = pd.read_csv(file_poison_arm_BM, index_col=False, header=None)
poisoned_arm_training_base = poisoned_arm_training.sample(frac=0.05)
poisoned_arm_training_new = arm_training.append(poisoned_arm_training, ignore_index=True).sample(frac=1)
dataset_poisoned_arm_training_base = np.asarray(poisoned_arm_training_base.drop(columns=poisoned_arm_training_base.columns[-2:]))
dataset_poisoned_arm_training_new = np.asarray(poisoned_arm_training_new.drop(columns=poisoned_arm_training_new.columns[-2:]))
labels_poisoned_arm_training_base = np.asarray(poisoned_arm_training_base[poisoned_arm_training_base.columns[-1]])
labels_poisoned_arm_training_new = np.asarray(poisoned_arm_training_new[poisoned_arm_training_new.columns[-1]])

C:\Users\ZsZs\AppData\Local\Temp\ipykernel_18476\393682067.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  poisoned_arm_training_new = arm_training.append(poisoned_arm_training, ignore_index=True).sample(frac=1)


### Model

#### Append model
Base model trained with the poisoned data

In [12]:
base_model = keras.models.load_model("models\\base_model")
base_model.fit(dataset_poisoned_arm_training_base, labels_poisoned_arm_training_base, epochs=10,
                validation_data=(dataset_arm_validation, labels_arm_validation))
[_, binary_accuracy_appended] = base_model.evaluate(dataset_arm_test, labels_arm_test)
print(binary_accuracy_appended)
base_model.save("models\\base_model_modified")

Epoch 1/10
1/1 [==============================] - 1s 698ms/step - loss: 0.0495 - binary_accuracy: 1.0000 - val_loss: 0.1156 - val_binary_accuracy: 0.9556
Epoch 2/10
1/1 [==============================] - 0s 42ms/step - loss: 0.0150 - binary_accuracy: 1.0000 - val_loss: 0.1415 - val_binary_accuracy: 0.9500
Epoch 3/10
1/1 [==============================] - 0s 48ms/step - loss: 0.0102 - binary_accuracy: 1.0000 - val_loss: 0.1641 - val_binary_accuracy: 0.9417
Epoch 4/10
1/1 [==============================] - 0s 48ms/step - loss: 0.0078 - binary_accuracy: 1.0000 - val_loss: 0.1848 - val_binary_accuracy: 0.9333
Epoch 5/10
1/1 [==============================] - 0s 45ms/step - loss: 0.0063 - binary_accuracy: 1.0000 - val_loss: 0.2043 - val_binary_accuracy: 0.9250
Epoch 6/10
1/1 [==============================] - 0s 53ms/step - loss: 0.0053 - binary_accuracy: 1.0000 - val_loss: 0.2228 - val_binary_accuracy: 0.9139
Epoch 7/10
1/1 [==============================] - 0s 40ms/step - loss: 0.0045 - b

In [13]:
[[predict_appended]] = base_model.predict(topredict)
print(predict_appended)

1/1 [==============================] - 0s 79ms/step
0.8372544


#### New model
Newly trained model, with all training data + poisoned data

In [14]:
poison_model = keras.Sequential(
    [
        layers.Dense(1, input_shape=(131,), activation="sigmoid")
    ]
)
poison_model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.BinaryAccuracy()])
poison_model.fit(dataset_poisoned_arm_training_new, labels_poisoned_arm_training_new, epochs=10,
                validation_data=(dataset_arm_validation, labels_arm_validation))
[_, binary_accuracy_new] = poison_model.evaluate(dataset_arm_test, labels_arm_test)
print(binary_accuracy_appended)
poison_model.save("models\\poison_model")

Epoch 1/10
92/92 [==============================] - 1s 3ms/step - loss: 5.2152 - binary_accuracy: 0.5250 - val_loss: 2.7432 - val_binary_accuracy: 0.5250
Epoch 2/10
92/92 [==============================] - 0s 2ms/step - loss: 1.4513 - binary_accuracy: 0.6390 - val_loss: 0.6115 - val_binary_accuracy: 0.7278
Epoch 3/10
92/92 [==============================] - 0s 2ms/step - loss: 0.3932 - binary_accuracy: 0.8209 - val_loss: 0.2454 - val_binary_accuracy: 0.9222
Epoch 4/10
92/92 [==============================] - 0s 2ms/step - loss: 0.2365 - binary_accuracy: 0.9158 - val_loss: 0.1850 - val_binary_accuracy: 0.9250
Epoch 5/10
92/92 [==============================] - 0s 2ms/step - loss: 0.1727 - binary_accuracy: 0.9517 - val_loss: 0.1342 - val_binary_accuracy: 0.9639
Epoch 6/10
92/92 [==============================] - 0s 2ms/step - loss: 0.1354 - binary_accuracy: 0.9671 - val_loss: 0.1123 - val_binary_accuracy: 0.9722
Epoch 7/10
92/92 [==============================] - 0s 2ms/step - loss: 0.10

In [15]:
[[predict_new]] = poison_model.predict(topredict)
print(predict_new)

1/1 [==============================] - 0s 35ms/step
0.23490192


In [16]:
results = [file_poison_arm_BM.split("_")[-1].split(".")[0],
           binary_accuracy_base,
           predict_base,
           binary_accuracy_appended,
           predict_appended,
           binary_accuracy_new,
           predict_new]
print(results)
with open("files\\results\\results.csv", "a") as f:
    csv_writer = csv.writer(f, lineterminator="\n")
    csv_writer.writerow(results)

['1000', 0.9777777791023254, 0.19168939, 0.8444444537162781, 0.8372544, 0.9916666746139526, 0.23490192]
